# Volume 3: MongoDB.
    <Jane Cox>
    <Math 403>
    <November 13 2018>

In [1]:
import re
import json
import numpy as np
from pymongo import MongoClient

## Problem 1

The file `trump.json` contains posts from http://www.twitter.com (tweets) over the course of an hour that have the key word "trump".
Each line in the file is a single JSON message that can be loaded with `json.loads()`.

Create a MongoDB database and initialize a collection in the database.
Use the collection's `delete_many()` method with an empy set as input to clear existing contents of the collection, then fill the collection one line at a time with the data from `trump.json`.
Check that your collection has 95,643 entries with its `count()` method.

In [5]:
client = MongoClient()
db1 = client.db1
collect = db1.collections2
collect.delete_many({})

with open('trump.json', 'r') as infile:
    for line in infile:
        collect.insert_one(json.loads(line))
    
collect.count()

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  # Remove the CWD from sys.path while we load stuff.


95643

## Problem 2

Query the Twitter collection for the following information.
- How many tweets include the word Russia? Use `re.IGNORECASE`.
- How many tweets came from one of the main continental US time zones? These are listed as `"Central Time (US & Canada)"`, `"Pacific Time (US & Canada)"`, `"Eastern Time (US & Canada)"`, and `"Mountain Time (US & Canada)"`.
- How often does each language occur? Construct a dictionary with each language and it’s frequency count.

In [6]:
! head trump.json

In [18]:
#Tweets containing Russia
russia_in = re.compile('russia',re.IGNORECASE)
russia_tweets = collect.find({'text':{'$regex':russia_in}})
print('There were', russia_tweets.count(), "tweets containing Russia.")

#Tweets from main continental US time
timezone = collect.find({'$or':[{'user.time_zone': 'Central Time (US & Canada)'},
                                {'user.time_zone': 'Pacific Time (US & Canada)'},
                               {'user.time_zone': 'Eastern Time (US & Canada)'},
                               {'user.time_zone': 'Mountain Time (US & Canada)'}]})
print('There were', timezone.count(), "tweets from US time zones.")

#Different Languages
languages = collect.distinct('user.lang')
diff_lang = {}

for lang in languages:
    sprache = collect.find({'user.lang':lang})
    count = sprache.count()
    diff_lang[lang] = count

diff_lang

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  after removing the cwd from sys.path.
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  # This is added back by InteractiveShellApp.init_path()


There were 5841 tweets containing Russia.
There were 33939 tweets from US time zones.


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.


{'af': 1,
 'ar': 95,
 'ca': 47,
 'cs': 13,
 'da': 49,
 'de': 1024,
 'el': 33,
 'en': 82984,
 'en-AU': 3,
 'en-GB': 93,
 'en-US': 1,
 'en-gb': 756,
 'es': 3496,
 'es-MX': 2,
 'eu': 1,
 'fa': 8,
 'fi': 36,
 'fr': 1612,
 'gl': 2,
 'he': 11,
 'hi': 1,
 'hr': 24,
 'hu': 6,
 'id': 95,
 'it': 588,
 'ja': 91,
 'ko': 15,
 'lv': 1,
 'nl': 389,
 'no': 19,
 'pl': 123,
 'pt': 752,
 'pt-PT': 1,
 'ro': 4,
 'ru': 779,
 'sk': 1,
 'sr': 6,
 'sv': 121,
 'th': 15,
 'tr': 615,
 'uk': 4,
 'vi': 6,
 'xx-lc': 5,
 'zh-CN': 9,
 'zh-Hans': 1,
 'zh-TW': 2,
 'zh-cn': 5,
 'zh-tw': 2}

## Problem 3

Query the Twitter collection from Problem 1 for the following information.
- What are the usernames of the 5 most popular (defined as having the most followers) tweeters? Don’t include repeats.
- Of the tweets containing at least 5 hashtags, sort the tweets by how early the 5th hashtag appears in the text. What is the earliest spot (character count) it appears?
- What are the coordinates of the tweet that came from the northernmost location? Use the latitude and longitude point in `"coordinates"`.

In [14]:
#find 5 most popular
followers = collect.find({'user.followers_count':{'$gt':1000000}})
followers.sort('user.followers_count', -1).limit(10)

print('Top 5 Twitter Usernames, by number of followers')
top_tweet = []
for person in followers:
    if person['user']['id'] not in top_tweet and len(top_tweet) < 5:
        top_tweet.append(person['user']['id'])
        print('\tFollowers:', person['user']['followers_count'],'\tUser:', person['user']['name'])

#Find earliest hashtags
hashtags = collect.find({'entities.hashtags.4':{'$exists': True}}).sort('entities.hashtags.4.indices', -1)
print("Earliest character location for hashtag:",
     hashtags[0]['entities']['hashtags'][4]['indices'][0])

#find northenmost tweet
polar = collect.find({'coordinates':{'$ne': None}}).sort('coordinates.coordinates.0', -1)
print('Northenmost tweeter:', polar[0]['coordinates']['coordinates'])


Top 5 Twitter Usernames, by number of followers
	Followers: 35481010 	User: CNN
	Followers: 17821353 	User: Reuters Top News
	Followers: 14663497 	User: Fox News
	Followers: 14023091 	User: Wall Street Journal
	Followers: 14017299 	User: TIME
Earliest character location for hashtag: 134
Northenmost tweeter: [28.99237342, 41.06803023]


## Problem 4

Clean the Twitter collection in the following ways.

- Get rid of the `"retweeted_status"` field in each tweet.
- Update every tweet from someone with at least 1000 followers to include a popular field whose value is True. Report the number of popular tweets.
- (OPTIONAL) The geographical coordinates used before in coordinates.coordinates are turned off for most tweets. But many more have a bounding box around the coordinates in the place field. Update every tweet without coordinates that contains a bounding box so that the coordinates contains the average value of the points that form the bounding box. Make the structure of coordinates the same as the others, so it contains coordinates with a longitude, latitude array and a type, the value of which should be ’Point’.

In [15]:
collect.update_many({'retweeted_stats':{'$exists':True}},{'$unset':{'retweeted_status':""}})


In [16]:
collect.update_many({'user.followers_count': {'$gte':1000}},{'$set':{'user.popular':True}})
popular = collect.find({'user.popular':{'$eq':True}})
print("Number of Popular Tweets:", popular.count())

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  This is separate from the ipykernel package so we can avoid doing imports until


Number of Popular Tweets: 32489
